In [2]:
def rnkOrdering(data_org, bad_col, wgt_col, score_col, n_bkts):
    if wgt_col is None:
        wgt_col = 'wt'
        data_org[wgt_col] = 1
    #data_org[score_col] = -1 * data_org[score_col]
    data = data_org.sort_values(by=[score_col]).reset_index()
    data["cum_wgt"] = data[wgt_col].cumsum()
    total = data[wgt_col].sum()
    bin_lvl = 0.1
    flag = 1
    for i in range(len(data)):
        if (data.loc[i, "cum_wgt"] >= total * bin_lvl) and (bin_lvl < 0.9):
            bin_lvl = bin_lvl + 0.1
            flag = flag + 1
            data.loc[i, "bucket"] = flag
        else:
            data.loc[i, "bucket"] = flag
    # data['bucket'] = pd.qcut(data[score_col], n_bkts)
    grouped = data.groupby('bucket', as_index=True)
    data["labels"] = data[bad_col] * data[wgt_col]
    agg1 = grouped[score_col].min()
    agg2 = grouped[score_col].max()
    agg3 = grouped[wgt_col].sum()
    agg4 = grouped["labels"].sum()
    agg5 = np.round(grouped[score_col].mean() * 100, 2)
    ret_tbl = pd.DataFrame({"min_score": agg1, "max_score": agg2, "total": agg3, "Bads": agg4, "Mean_Score": agg5})
    ret_tbl["Goods"] = ret_tbl["total"] - ret_tbl["Bads"]
    ret_tbl["Bad_Rate"] = np.round(ret_tbl["Bads"] / ret_tbl["total"], 4) * 100
    ret_tbl = ret_tbl.sort_index(axis=0, ascending=False)
    ret_tbl.index = range(n_bkts)

    ret_tbl["Cum_Bad"] = np.round((ret_tbl.Bads / ret_tbl.Bads.sum()).cumsum(), 4) * 100
    ret_tbl["Cum_Good"] = np.round((ret_tbl.Goods / ret_tbl.Goods.sum()).cumsum(), 4) * 100
    ret_tbl['KS'] = np.abs(ret_tbl["Cum_Bad"] - ret_tbl["Cum_Good"])

    gini = ret_tbl["Cum_Bad"][0] * ret_tbl["Cum_Good"][0] / (2 * 100 * 100)
    for i in range(1, n_bkts):
        gini += (ret_tbl["Cum_Bad"][i] + ret_tbl["Cum_Bad"][i - 1]) * (ret_tbl["Cum_Good"][i] -
                                                                       ret_tbl["Cum_Good"][i - 1]) / (2 * 100 * 100)
    gini = np.round(2. * gini - 1., 4) * 100

    ks = np.round(ret_tbl.KS.max(), 2)

    coltitles = ["min_score", "max_score", "Mean_Score", "total", "Goods", "Bads", "Bad_Rate", "Cum_Bad", "Cum_Good", "KS"]

    return ret_tbl[coltitles], {"Gini": gini, "KS": ks}

In [2]:
import pandas as pd
import numpy as np
data_org = pd.read_csv("/home/mithun/projectFiles/modelPerformanceTesting/Scored_oot_file_1.csv/part-00000-82e33775-5470-4d4c-b79e-7d91bd9ea3d5-c000.csv")
data_org

,weight,bad,open_oblig_cnt_6m,sum_dpd_now_minus10_adj,sum_dpd_now_burn_off_minus10_ad,never_over90_dpd_flg,lender_cnt,annual_sales_amt,pct_orig_amt_w_indep_lender,subborrower_cnt,...,L_primarypaynetassettypecd_Missing,unique_id,Y,pred,wt,Y_index_HGCHFmj45XQada6fAdyYOrVtTJvB7sX8,prediction,__predicted_Y_index_HGCHFmj45XQada6fAdyYOrVtTJvB7sX8__,probability_0,probability_1
0,0.998118,0,3.0,-70.0,-30.0,1.0,3.0,1632.0,58.0,0.0,...,0,165141,0,0.060203,0.998118,0.0,0.0,0,0.989609,0.010391
1,0.019507,1,67.0,-1220.0,-344.6,0.0,16.0,13200.0,42.0,5.0,...,0,165142,1,0.065943,0.019507,1.0,0.0,0,0.978841,0.021159
2,0.019507,1,67.0,-1220.0,-344.6,0.0,16.0,13200.0,42.0,5.0,...,0,165142,0,0.065943,0.980493,0.0,0.0,0,0.978841,0.021159
3,0.980493,0,67.0,-1220.0,-344.6,0.0,16.0,13200.0,42.0,5.0,...,0,165142,1,0.065943,0.019507,1.0,0.0,0,0.972222,0.027778
4,0.980493,0,67.0,-1220.0,-344.6,0.0,16.0,13200.0,42.0,5.0,...,0,165142,0,0.065943,0.980493,0.0,0.0,0,0.972222,0.027778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21648,0.989297,0,2.0,91.0,101.0,0.0,2.0,61.0,0.0,0.0,...,0,107387,1,0.132932,0.010703,1.0,0.0,0,0.986120,0.013880
21649,0.989297,0,2.0,91.0,101.0,0.0,2.0,61.0,0.0,0.0,...,0,107387,0,0.132932,0.989297,0.0,0.0,0,0.986120,0.013880
21650,0.013586,1,1.0,-20.0,-10.0,1.0,2.0,4590.0,0.0,0.0,...,0,107409,1,0.038080,0.013586,1.0,0.0,0,0.991911,0.008089
21651,0.013586,1,1.0,-20.0,-10.0,1.0,2.0,4590.0,0.0,0.0,...,0,107409,0,0.038080,0.986414,0.0,0.0,0,0.991911,0.008089


In [4]:
wgt_col = "wt"
score_col = "probability_1"
bad_col = "Y"
if wgt_col is None:
    wgt_col = 'wt'
    data_org[wgt_col] = 1
#data_org[score_col] = -1 * data_org[score_col]
data = data_org.sort_values(by=[score_col]).reset_index()
data["cum_wgt"] = data[wgt_col].cumsum()
total = data[wgt_col].sum()
bin_lvl = 0.1
flag = 1
for i in range(len(data)):
    if (data.loc[i, "cum_wgt"] >= total * bin_lvl) and (bin_lvl < 0.9):
        bin_lvl = bin_lvl + 0.1
        flag = flag + 1
        data.loc[i, "bucket"] = flag
    else:
        data.loc[i, "bucket"] = flag
# data['bucket'] = pd.qcut(data[score_col], n_bkts)
grouped = data.groupby('bucket', as_index=True)
data["labels"] = data[bad_col] * data[wgt_col]
data

,index,weight,bad,open_oblig_cnt_6m,sum_dpd_now_minus10_adj,sum_dpd_now_burn_off_minus10_ad,never_over90_dpd_flg,lender_cnt,annual_sales_amt,pct_orig_amt_w_indep_lender,...,pred,wt,Y_index_HGCHFmj45XQada6fAdyYOrVtTJvB7sX8,prediction,__predicted_Y_index_HGCHFmj45XQada6fAdyYOrVtTJvB7sX8__,probability_0,probability_1,cum_wgt,bucket,labels
0,5772,0.001014,1,1.0,-50.0,-10.0,1.0,2.0,-999999.0,100.0,...,0.037111,0.001014,1.0,0.0,0,0.998443,0.001557,0.001014,1.0,0.001014
1,5773,0.001014,1,1.0,-50.0,-10.0,1.0,2.0,-999999.0,100.0,...,0.037111,0.998986,0.0,0.0,0,0.998443,0.001557,1.000000,1.0,0.000000
2,18914,0.000188,1,2.0,-30.0,-20.0,1.0,3.0,14030.0,62.0,...,0.036773,0.000188,1.0,0.0,0,0.998235,0.001765,1.000188,1.0,0.000188
3,18915,0.000188,1,2.0,-30.0,-20.0,1.0,3.0,14030.0,62.0,...,0.036773,0.999812,0.0,0.0,0,0.998235,0.001765,2.000000,1.0,0.000000
4,12742,0.000274,1,12.0,-220.0,-100.0,0.0,12.0,12000.0,10.0,...,0.037002,0.000274,1.0,0.0,0,0.998208,0.001792,2.000274,1.0,0.000274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21648,15597,1.000000,1,1.0,-10.0,-10.0,1.0,1.0,-999999.0,0.0,...,0.050528,1.000000,1.0,1.0,1,0.026788,0.973212,12065.000000,10.0,1.000000
21649,15596,1.000000,1,1.0,-10.0,-10.0,1.0,1.0,-999999.0,0.0,...,0.050528,1.000000,1.0,1.0,1,0.026788,0.973212,12066.000000,10.0,1.000000
21650,15595,1.000000,1,1.0,-10.0,-10.0,1.0,1.0,-999999.0,0.0,...,0.050528,1.000000,1.0,1.0,1,0.026788,0.973212,12067.000000,10.0,1.000000
21651,15607,1.000000,1,1.0,-10.0,-10.0,1.0,1.0,-999999.0,0.0,...,0.050528,1.000000,1.0,1.0,1,0.026788,0.973212,12068.000000,10.0,1.000000


In [6]:
n_bkts = 10
agg1 = grouped[score_col].min()
agg2 = grouped[score_col].max()
agg3 = grouped[wgt_col].sum()
agg4 = grouped["labels"].sum()
agg5 = np.round(grouped[score_col].mean() * 100, 2)
ret_tbl = pd.DataFrame({"min_score": agg1, "max_score": agg2, "total": agg3, "Bads": agg4, "Mean_Score": agg5})
ret_tbl["Goods"] = ret_tbl["total"] - ret_tbl["Bads"]
ret_tbl["Bad_Rate"] = np.round(ret_tbl["Bads"] / ret_tbl["total"], 4) * 100
ret_tbl = ret_tbl.sort_index(axis=0, ascending=False)
ret_tbl.index = range(n_bkts)

ret_tbl["Cum_Bad"] = np.round((ret_tbl.Bads / ret_tbl.Bads.sum()).cumsum(), 4) * 100
ret_tbl["Cum_Good"] = np.round((ret_tbl.Goods / ret_tbl.Goods.sum()).cumsum(), 4) * 100
ret_tbl['KS'] = np.abs(ret_tbl["Cum_Bad"] - ret_tbl["Cum_Good"])

gini = ret_tbl["Cum_Bad"][0] * ret_tbl["Cum_Good"][0] / (2 * 100 * 100)
for i in range(1, n_bkts):
    gini += (ret_tbl["Cum_Bad"][i] + ret_tbl["Cum_Bad"][i - 1]) * (ret_tbl["Cum_Good"][i] -
                                                                   ret_tbl["Cum_Good"][i - 1]) / (2 * 100 * 100)
gini = np.round(2. * gini - 1., 4) * 100

ks = np.round(ret_tbl.KS.max(), 2)

coltitles = ["min_score", "max_score", "Mean_Score", "total", "Goods", "Bads", "Bad_Rate", "Cum_Bad", "Cum_Good", "KS"]
{"Gini": gini, "ks": ks}

{'Gini': 87.32, 'ks': 76.95}